In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
# from lib.notebook_utils import setup_multiprocessing_for_notebook
# setup_multiprocessing_for_notebook()

import os
import logging
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

from solution import (
    prepare_dataset,
    build_baseline_setup,
    build_deepspeed_setup,
    build_fsdp_setup,
    run_training_session,
)

logger = logging.getLogger(__name__)

# _ = prepare_dataset() # подготовка данных (один раз)
print('-----------------------')
print("CUDA_VISIBLE_DEVICES =", os.getenv("CUDA_VISIBLE_DEVICES"))
print("WANDB_PROJECT = ", os.getenv("WANDB_PROJECT"))
print("WANDB_API_KEY =", f'{os.getenv("WANDB_API_KEY")[:3]}...{os.getenv("WANDB_API_KEY")[-3:]}')

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!accelerate launch \
  --num_processes 2 --num_machines 1 --gpu_ids 2,3 \
  --mixed_precision bf16 \
  train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --batch-size 16 \
    --grad-accum 4 \
    --timeout 1800 \
    --run-name "ds1_accel_2gpus" \
    --torch-compile false

In [ ]:
!accelerate launch --num_processes 1 --gpu_ids 1 ../mini.py


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading dataset shards: 100%|█████████████████| 27/27 [00:00<00:00, 3990.35it/s]
Training samples:   1940063
Validation samples: 5000
/app/lib/training.py:87: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.10
wandb: Run data is saved locally in /app/hw2_parallel_pretrain/wandb/run-20251103_065905-v771ak37
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bs16_ga4_lr5e-05_adamw_torch_baseline_fc1
wandb: ⭐️ View 

In [2]:
!CUDA_VISIBLE_DEVICES=1 torchrun --nproc_per_node=1 --master_port=29501 ../train_distributed.py \
    --mode baseline \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "baseline_test"

# ~ 29.5гб на 1 GPU, загрузка полная (95-100%)
# Успело пройти 1175 шагов обучения

2025-11-03 06:32:15,748 [INFO] __main__: ============================================================
2025-11-03 06:32:15,748 [INFO] __main__: Запуск обучения в режиме: baseline
2025-11-03 06:32:15,748 [INFO] __main__: ============================================================
2025-11-03 06:32:15,748 [INFO] __main__: World size: 1
2025-11-03 06:32:15,749 [INFO] __main__: CUDA_VISIBLE_DEVICES: 1
2025-11-03 06:32:15,749 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-03 06:32:15,749 [INFO] __main__: Создание baseline setup (single GPU)
--------------------------------
{'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 4, 'bf16': True, 'report_to': 'wandb'}
--------------------------------
Namespace(mode='baseline', deepspeed_stage=2, offload_optimizer=False, offload_params=False, batch_size=16, grad_accum=4, max_steps=None, timeout=1800, run_name='baseline_test', no_wandb=False, data_dir='/app/output_dir', master_port=29500)
--------------------------------
Loa

___

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 torchrun --nproc_per_node=2 --master_port=29502 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 1 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "deep_speed_setup_1"

# ~ по 53гб на каждый GPU, загрузка полная (95-100%)

2025-11-02 18:36:29,438 [INFO] __main__: ============================================================
2025-11-02 18:36:29,438 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:36:29,438 [INFO] __main__: ============================================================
2025-11-02 18:36:29,879 [INFO] __main__: ============================================================
2025-11-02 18:36:29,879 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:36:29,879 [INFO] __main__: ============================================================
2025-11-02 18:36:29,879 [INFO] __main__: World size: 2
2025-11-02 18:36:29,879 [INFO] __main__: CUDA_VISIBLE_DEVICES: 0,1
2025-11-02 18:36:29,879 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-02 18:36:29,879 [INFO] __main__: Создание DeepSpeed setup (stage 1)
Loading dataset shards: 100%|█████████████████| 27/27 [00:00<00:00, 2788.49it/s]
Training samples:   1940063
Validation samples: 5000
Loading dataset shards: 100%|████

___

In [2]:
!CUDA_VISIBLE_DEVICES=2,3 torchrun --nproc_per_node=2 --master_port=29501 ../train_distributed.py \
    --mode deepspeed \
    --deepspeed-stage 2 \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "deep_speed_setup_2"

# ~ по 50гб на каждый GPU, загрузка полная (95-100%)

2025-11-02 18:35:36,764 [INFO] __main__: ============================================================
2025-11-02 18:35:36,764 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:35:36,764 [INFO] __main__: ============================================================
2025-11-02 18:35:36,764 [INFO] __main__: World size: 2
2025-11-02 18:35:36,764 [INFO] __main__: CUDA_VISIBLE_DEVICES: 2,3
2025-11-02 18:35:36,764 [INFO] __main__: WANDB_PROJECT: llm_hw2-aylesnov
2025-11-02 18:35:36,764 [INFO] __main__: Создание DeepSpeed setup (stage 2)
2025-11-02 18:35:37,066 [INFO] __main__: ============================================================
2025-11-02 18:35:37,066 [INFO] __main__: Запуск обучения в режиме: deepspeed
2025-11-02 18:35:37,067 [INFO] __main__: ============================================================
Loading dataset shards: 100%|█████████████████| 27/27 [00:00<00:00, 2488.00it/s]
Training samples:   1940063
Validation samples: 5000
Loading dataset shards: 100%|████

___

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 torchrun --nproc_per_node=2 ../train_distributed.py \
    --mode fsdp \
    --batch-size 16 \
    --grad-accum 4 \
    --run-name "fsdp"